In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
path = '/home/angelmc/MyFirstDSProject'

In [4]:
df = pd.read_csv(path+'/data/Dataset Endeudamiento Crediticio.csv',sep=';')

In [5]:
df.head()

,ID,Default,Prct_uso_tc,Edad,Nro_prestao_retrasados,Prct_deuda_vs_ingresos,Mto_ingreso_mensual,Nro_prod_financieros_deuda,Nro_retraso_60dias,Nro_creditos_hipotecarios,Nro_retraso_ultm3anios,Nro_dependiente
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [7]:
# Definiendo el target
target = 'Default'
# Definiendo las columnas que servirán para predecir el target
columnas = list(set(df.columns) - set([target]))
coltotal = columnas + [target]

In [8]:
df_miss = pd.DataFrame({'count': df.isnull().sum(), 'percentage': df.isnull().mean()*100})
df_miss.to_csv(path+'/reports/1st_missing_report.csv')

In [12]:
def setTratamientoCotasSup(base,var_T,q):
    p=base[var_T].quantile(q=q)
    qq=(q+0.05) if (q+0.05<1) else q
    base[f"{var_T}"]=base[var_T].apply(lambda x: p if x>=p else x)
def setTratamientoCotasInf(base,var_T,q):
    p=base[var_T].quantile(q=q)
    qq=(q-0.05) if (q-0.05>0) else q
    base[f"{var_T}"]=base[var_T].apply(lambda x: p if x<=p else x)

In [13]:
setTratamientoCotasSup(df,'Edad',0.99)
setTratamientoCotasInf(df,'Edad',0.01)

setTratamientoCotasSup(df,'Mto_ingreso_mensual',0.99)
setTratamientoCotasInf(df,'Mto_ingreso_mensual',0.02)

setTratamientoCotasSup(df,'Prct_uso_tc',0.97)

setTratamientoCotasSup(df,'Prct_deuda_vs_ingresos',0.80)

In [17]:
col_nulls = ['Mto_ingreso_mensual','Nro_dependiente']

def setTratamientoMissingsVarContinua(base,var_T):
    imputer=base[var_T].mode().values[0]
    base[f"{var_T}"]=base[var_T].fillna(imputer)
    print(f"{var_T}(Mode: {imputer})")

for i in col_nulls:
    setTratamientoMissingsVarContinua(df,i)

Mto_ingreso_mensual(Mode: 5000.0)
Nro_dependiente(Mode: 0.0)


In [18]:
df_miss = pd.DataFrame({'count': df.isnull().sum(), 'percentage': df.isnull().mean()*100})
df_miss.to_csv(path+'/reports/2nd_missing_report.csv')

In [20]:
print('hola mundo')

hola mundo
